In [4]:
import pandas as pd
import util1
import util2

In [ ]:
util1.clean_and_save_sales_data()
util1.unpivot_combine_and_save_sales_data()

In [3]:
util2.clean_and_save_comments_and_opening_closure_data()
util2.combine_and_save_sales_comments()
util2.combine_and_save_store_openings_and_closures()

In [ ]:
sales_data = pd.read_csv('./data/preprocessed/sales_data.csv')
sales_data.head(5)

In [6]:
monthly_sales_comments = pd.read_excel('./data/preprocessed/monthly_sales_comments.xlsx')
monthly_sales_comments.head(5)

,Period,Term,Information Type,Information
0,August 2017,September 2016 to August 2017,Sales Comments,August 2017 same-store sales decreased by 3.4%...
1,July 2017,September 2016 to August 2017,Sales Comments,July 2017 same-store sales increased by 3.7% y...
2,June 2017,September 2016 to August 2017,Sales Comments,June 2017 same-store sales increased by 4.1% y...
3,May 2017,September 2016 to August 2017,Sales Comments,May 2017 same-store sales increased by 2.4% ye...
4,April 2017,September 2016 to August 2017,Sales Comments,April 2017 same-store sales increased by 6.2% ...


In [8]:
monthly_openings_closures = pd.read_csv('./data/preprocessed/monthly_openings_and_closures.csv')
monthly_openings_closures.head(5)

,Period,Openings,Closures,Term
0,August 2017,0.0,2.0,September 2016 to August 2017
1,July 2017,0.0,1.0,September 2016 to August 2017
2,June 2017,0.0,0.0,September 2016 to August 2017
3,May 2017,0.0,1.0,September 2016 to August 2017
4,April 2017,6.0,2.0,September 2016 to August 2017
